In [4]:
import csv
with open("best_results.csv") as fp:
    reader = csv.reader(fp)
    best_results = []
    for i, row in enumerate(reader):
        if i == 0:
            continue
        best_results.append(map(float, row))
        
reachable = [(lng, lat) for lat, lng, travel_time in best_results if travel_time <= 180.0 * 60.0]

In [5]:
from collections import defaultdict

import numpy as np
from scipy.spatial import Delaunay
from shapely.geometry import mapping, MultiLineString, MultiPoint, Point, Polygon
from shapely.ops import polygonize, unary_union

In [6]:
coords = np.array(reachable)
tri = Delaunay(coords)

In [7]:
simplices = tri.simplices  # pylint: disable=no-member

In [8]:
tri_coords = coords[simplices]

In [9]:
tri_coords

array([[[-85.8827506 ,  38.302869  ],
        [-85.87881362,  38.32617253],
        [-85.94176033,  38.00180457]],

       [[-85.8787968 ,  38.28559369],
        [-85.8827506 ,  38.302869  ],
        [-85.8994469 ,  38.1696455 ]],

       [[-85.74493539,  37.9654092 ],
        [-85.8982444 ,  38.0093875 ],
        [-85.9038693 ,  38.0079034 ]],

       ...,

       [[-85.7142716 ,  38.2572575 ],
        [-85.7142798 ,  38.2572433 ],
        [-85.7142365 ,  38.2572271 ]],

       [[-85.7062327 ,  38.2535349 ],
        [-85.7060656 ,  38.2533343 ],
        [-85.7060662 ,  38.253345  ]],

       [[-85.7060656 ,  38.2533343 ],
        [-85.7062327 ,  38.2535349 ],
        [-85.7062084 ,  38.2531314 ]]])

In [12]:
tri_coords.shape
alpha = 300
inv_alpha = 1.0 / alpha

In [13]:
# Lengths of sides of triangle
a = np.sqrt(
    np.sum(np.square(tri_coords[:, 0, :] - tri_coords[:, 1, :]), 1))
b = np.sqrt(
    np.sum(np.square(tri_coords[:, 1, :] - tri_coords[:, 2, :]), 1))
c = np.sqrt(
    np.sum(np.square(tri_coords[:, 2, :] - tri_coords[:, 0, :]), 1))
# Semiperimeter of triangle
s = (a + b + c) * 0.5
# Area of triangle by Heron's formula
area = s * (s - a) * (s - b) * (s - c)
circumradius = a * b * c / (4.0 * np.sqrt(area))
# Filter based on the circumradius value compared with the alpha value:
# 1. circumradius is less than inv_alpha -> triangle is included in shape
# 2. circumradius is finite and >= inv_alpha -> triangle should not be included in shape
# 3. circumradius is NaN or infinite -> ignore (Due to how we handle holes below, it's
#    simpler to just skip these triangles altogether.)
included_triangles = circumradius < inv_alpha
hole_triangles = np.all(
    [np.isfinite(circumradius), circumradius >= inv_alpha], 0)

In [14]:
internal_faces_count = defaultdict(int)
for ia, ib, ic in simplices[included_triangles]:
    for i, j in [(ia, ib), (ib, ic), (ic, ia)]:
        ordered_indices = (i, j) if i < j else (j, i)
        internal_faces_count[ordered_indices] += 1

In [17]:
simplices[included_triangles]

array([[ 11383, 122589, 122387],
       [ 11383,  13041, 122589],
       [122470,  11383, 122387],
       ...,
       [ 75558,  75884,  82978],
       [113855,  83283,  96582],
       [ 83283, 113855,  78227]], dtype=int32)

In [18]:
coords.shape

(125332, 2)

In [19]:
coords[:10]

array([[-85.864426 ,  38.281571 ],
       [-85.8698606,  38.2836724],
       [-85.8087205,  38.2952647],
       [-85.8083695,  38.294866 ],
       [-85.8191283,  38.3436285],
       [-85.8190999,  38.3437967],
       [-85.8144372,  38.3404756],
       [-85.8140188,  38.3398879],
       [-85.8206762,  38.3404473],
       [-85.820755 ,  38.3403844]])

In [20]:
coords[0]

array([-85.864426,  38.281571])

In [23]:
simplices

array([[  7692, 122260, 122660],
       [122237,   7692,  25838],
       [125198,  20027,  15986],
       ...,
       [ 75558,  75884,  82978],
       [113855,  83283,  96582],
       [ 83283, 113855,  78227]], dtype=int32)

In [24]:
coords[[2, 3]]

array([[-85.8087205,  38.2952647],
       [-85.8083695,  38.294866 ]])

In [25]:
coords[2]

array([-85.8087205,  38.2952647])

In [26]:
coords[3]

array([-85.8083695,  38.294866 ])